In [ ]:
from datetime import datetime

class TaskNode:
    def __init__(self, task_id, description, priority, entry_time, skill_required, doctor_assigned, day, duration=30):
        self.task_id = task_id
        self.description = description
        self.priority = priority
        self.entry_time = datetime.strptime(entry_time, "%H:%M")
        self.skill_required = skill_required
        self.doctor_assigned = doctor_assigned
        self.day = day
        self.duration = duration  # Task duration in minutes
        self.status = "Pending"  # Track task status
        self.next = None
        
        # Calculated score for positioning in queue
        self.position_score = self.calculate_position_score()
    
    def calculate_position_score(self):
        # Convert time to minutes for easier calculation (0-1440 minutes in a day)
        time_in_minutes = self.entry_time.hour * 60 + self.entry_time.minute
        
        # Priority score: inverse of priority (1-5) multiplied by 100 for weighting
        # Lower priority number means higher importance (1 is highest priority)
        priority_score = (6 - self.priority) * 100
        
        # Time factor: earlier times get higher scores
        # We use 1440 (minutes in a day) minus the current time to give higher scores to earlier times
        time_factor = 1440 - time_in_minutes
        
        # Combined score: weighted sum of priority and time
        # Higher score = higher position in queue
        return priority_score * 0.7 + time_factor * 0.3

class PriorityQueue:
    def __init__(self):
        self.head = None
        self.task_count = 0
    
    def enqueue(self, task_node):
        """Add a task to the queue based on its position score"""
        self.task_count += 1
        
        # If queue is empty or new task has higher score than head
        if self.head is None or self.head.position_score < task_node.position_score:
            task_node.next = self.head
            self.head = task_node
        else:
            # Find the right position to insert based on position_score
            temp = self.head
            while temp.next and temp.next.position_score >= task_node.position_score:
                temp = temp.next
            task_node.next = temp.next
            temp.next = task_node
    
    def dequeue(self):
        """Remove and return the highest priority task"""
        if self.head is None:
            print("No tasks in queue!")
            return None
        self.task_count -= 1
        highest_priority_task = self.head
        self.head = self.head.next
        return highest_priority_task
    
    def show_tasks(self, day=None, status=None):
        """Display tasks filtered by day and/or status"""
        temp = self.head
        print(f"\nTasks{' scheduled for ' + day if day else ''}{' with status ' + status if status else ''}:")
        found = False
        while temp:
            if (not day or temp.day.lower() == day.lower()) and (not status or temp.status == status):
                print(f"Task ID {temp.task_id}: {temp.description}")
                print(f"Priority: {temp.priority}, Position Score: {temp.position_score:.2f}")
                print(f"Duration: {temp.duration} mins, Status: {temp.status}")
                print(f"Assigned to Dr. {temp.doctor_assigned}")
                print(f"Scheduled for: {temp.day} at {temp.entry_time.strftime('%H:%M')}\n")
                found = True
            temp = temp.next
        if not found:
            print("No matching tasks found.")

class Robot:
    def __init__(self, name, skills, availability=True):
        self.name = name
        self.skills = skills
        self.availability = availability
        self.completed_tasks = []
        self.current_task = None

    def can_perform_task(self, skill_required):
        return self.availability and skill_required in self.skills

    def assign_task(self, task):
        self.current_task = task
        self.availability = False
        task.status = "In Progress"

    def complete_task(self):
        if self.current_task:
            self.current_task.status = "Completed"
            self.completed_tasks.append(self.current_task)
            self.current_task = None
            self.availability = True

class Scheduler:
    def __init__(self):
        self.tasks = PriorityQueue()
        self.robots = []
        self.doctors = ["Dr. KP", "Dr. Sasank"]
        self.task_counter = 0
        self.skills_available = ["surgery", "checkup", "testing", "emergency", "consultation"]
    
    def add_robot(self, name, skills):
        for skill in skills:
            if skill not in self.skills_available:
                print(f"Warning: {skill} is not in recognized skills list")
        self.robots.append(Robot(name, skills))
        print(f"Robot {name} added with skills: {skills}")
    
    def remove_robot(self, name):
        for i, robot in enumerate(self.robots):
            if robot.name == name:
                if not robot.availability:
                    print(f"Warning: Robot {name} is currently busy with a task")
                    return
                self.robots.pop(i)
                print(f"Robot {name} removed!")
                return
        print("Robot not found!")
    
    def add_doctor(self, name):
        if name not in self.doctors:
            self.doctors.append(name)
            print(f"Doctor {name} added!")
    
    def remove_doctor(self, name):
        tasks_affected = False
        temp = self.tasks.head
        while temp:
            if temp.doctor_assigned == name:
                tasks_affected = True
                break
            temp = temp.next
        
        if tasks_affected:
            print(f"Cannot remove {name}. They have pending tasks!")
            return
            
        if name in self.doctors:
            self.doctors.remove(name)
            print(f"Doctor {name} removed!")
        else:
            print("Doctor not found!")
    
    def add_task(self, description, priority, entry_time, skill_required, doctor_assigned, day, duration=30):
        if doctor_assigned not in self.doctors:
            print("Error: Doctor not found!")
            return
        if skill_required not in self.skills_available:
            print(f"Warning: {skill_required} is not in recognized skills list")
        try:
            datetime.strptime(entry_time, "%H:%M")
        except ValueError:
            print("Error: Invalid time format! Use HH:MM")
            return
        if not 1 <= priority <= 5:
            print("Error: Priority must be between 1 and 5")
            return
            
        self.task_counter += 1
        new_task = TaskNode(self.task_counter, description, priority, entry_time, 
                           skill_required, doctor_assigned, day, duration)
        self.tasks.enqueue(new_task)
        print(f"Task {self.task_counter} added successfully with position score: {new_task.position_score:.2f}")
    
    def process_task(self):
        task = self.tasks.dequeue()
        if task:
            print(f"Processing Task {task.task_id}: {task.description}")
            print(f"Priority: {task.priority}, Position Score: {task.position_score:.2f}")
            print(f"Required Skill: {task.skill_required}")
            task.status = "Completed"
    
    def assign_robot_to_task(self):
        task = self.tasks.dequeue()
        if not task:
            print("No tasks to assign!")
            return
            
        available_robots = [robot for robot in self.robots 
                          if robot.can_perform_task(task.skill_required)]
        
        if available_robots:
            robot = available_robots[0]  # Choose first available robot
            robot.assign_task(task)
            print(f"{robot.name} has been assigned to Task {task.task_id}")
            return
            
        print(f"No available robot with skill '{task.skill_required}'. Task re-added to queue.")
        self.tasks.enqueue(task)
    
    def show_day_schedule(self, day):
        self.tasks.show_tasks(day=day)
    
    def get_queue_stats(self):
        print(f"\nQueue Statistics:")
        print(f"Total tasks in queue: {self.tasks.task_count}")
        print(f"Available robots: {sum(1 for robot in self.robots if robot.availability)}")
        print(f"Total doctors: {len(self.doctors)}")

def main():
    scheduler = Scheduler()
    
    # Initialize with some robots
    scheduler.add_robot("Robo1", ["surgery", "emergency"])
    scheduler.add_robot("Robo2", ["checkup", "testing"])
    scheduler.add_robot("Robo3", ["testing", "consultation"])
    
    while True:
        print("\nHospital Task Scheduler Menu:")
        print("1. Add Task")
        print("2. Process Task")
        print("3. Assign Robot to Task")
        print("4. Show Schedule for a Day")
        print("5. Add/Remove Robot")
        print("6. Add/Remove Doctor")
        print("7. Show Queue Statistics")
        print("8. Exit")
        
        try:
            choice = input("Choose an option: ")
            
            if choice == "1":
                description = input("Enter Task Description: ")
                while True:
                    try:
                        priority = int(input("Enter Priority (1-5): "))
                        if 1 <= priority <= 5:
                            break
                        print("Priority must be between 1 and 5")
                    except ValueError:
                        print("Please enter a valid number")
                
                entry_time = input("Enter Task Entry Time (HH:MM format): ")
                print("Available Skills:", ", ".join(scheduler.skills_available))
                skill_required = input("Enter Required Skill: ")
                print("Available Doctors:", ", ".join(scheduler.doctors))
                doctor_assigned = input("Assign Doctor: ")
                day = input("Enter Day (Monday-Sunday): ")
                duration = int(input("Enter Task Duration (minutes): "))
                
                scheduler.add_task(description, priority, entry_time, skill_required,
                                 doctor_assigned, day, duration)
            
            elif choice == "2":
                scheduler.process_task()
            
            elif choice == "3":
                scheduler.assign_robot_to_task()
            
            elif choice == "4":
                day = input("Enter Day (Monday-Sunday): ")
                scheduler.show_day_schedule(day)
            
            elif choice == "5":
                action = input("(a) Add Robot or (r) Remove Robot? ").lower()
                if action == "a":
                    name = input("Enter Robot Name: ")
                    print("Available Skills:", ", ".join(scheduler.skills_available))
                    skills = input("Enter Skills (comma separated): ").split(",")
                    scheduler.add_robot(name, [skill.strip() for skill in skills])
                elif action == "r":
                    name = input("Enter Robot Name: ")
                    scheduler.remove_robot(name)
            
            elif choice == "6":
                action = input("(a) Add Doctor or (r) Remove Doctor? ").lower()
                if action == "a":
                    name = input("Enter Doctor Name: ")
                    scheduler.add_doctor(name)
                elif action == "r":
                    name = input("Enter Doctor Name: ")
                    scheduler.remove_doctor(name)
            
            elif choice == "7":
                scheduler.get_queue_stats()
            
            elif choice == "8":
                print("Thank you for using Hospital Task Scheduler!")
                break
            
            else:
                print("Invalid option! Please select from 1-8.")
                
        except ValueError as e:
            print(f"Error: {e}")
            print("Please try again with valid input.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            print("Please try again.")

if __name__ == "__main__":
    main()

Robot Robo1 added with skills: ['surgery', 'emergency']
Robot Robo2 added with skills: ['checkup', 'testing']
Robot Robo3 added with skills: ['testing', 'consultation']

Hospital Task Scheduler Menu:
1. Add Task
2. Process Task
3. Assign Robot to Task
4. Show Schedule for a Day
5. Add/Remove Robot
6. Add/Remove Doctor
7. Show Queue Statistics
8. Exit


Choose an option:  1
Enter Task Description:  headache
Enter Priority (1-5):  5
Enter Task Entry Time (HH:MM format):  10:30


Available Skills: surgery, checkup, testing, emergency, consultation


Enter Required Skill:  checkup


Available Doctors: Dr. KP, Dr. Sasank


...

Available Doctors: Dr. KP, Dr. Sasank


Assign Doctor:  Dr. Sasank
Enter Day (Monday-Sunday):  Sunday
Enter Task Duration (minutes):  150


Task 2 added successfully with position score: 593.00

Hospital Task Scheduler Menu:
1. Add Task
2. Process Task
3. Assign Robot to Task
4. Show Schedule for a Day
5. Add/Remove Robot
6. Add/Remove Doctor
7. Show Queue Statistics
8. Exit


Choose an option:  4
Enter Day (Monday-Sunday):  Sunday



Tasks scheduled for Sunday:
Task ID 2: heart surgery
Priority: 1, Position Score: 593.00
Duration: 150 mins, Status: Pending
Assigned to Dr. Dr. Sasank
Scheduled for: Sunday at 10:30

Task ID 1: headache
Priority: 5, Position Score: 313.00
Duration: 12 mins, Status: Pending
Assigned to Dr. Dr. KP
Scheduled for: Sunday at 10:30


Hospital Task Scheduler Menu:
1. Add Task
2. Process Task
3. Assign Robot to Task
4. Show Schedule for a Day
5. Add/Remove Robot
6. Add/Remove Doctor
7. Show Queue Statistics
8. Exit
